# ビジネスケース　Audiobooks

### データの読み込み

In [19]:
import numpy as np
import tensorflow as tf

# 前処理にはsklearnライブラリを使っていきます
from sklearn import preprocessing


# データを読み込みます
raw_csv_data = np.loadtxt("../../data/Audiobooks-data.csv", delimiter=",")

# 最初のIDの列と、ターゲットである最後の列を取り除いた上で変数に代入します
unscaled_inputs_all = raw_csv_data[:, 1:-1]

# 最後の列のデータをターゲットとして変数に代入していきます
targets_all = raw_csv_data[:, -1]

### データセットのバランシング

In [20]:
# ターゲットの数を数えます
num_one_targets = int(np.sum(targets_all))

# 数を数えるための変数を定義します
zero_targets_counter = 0

# バランスを取る上で不要なデータを入れるためのリストを定義します
indices_to_remove = []

# ターゲットの数を数え、その数を超える0のデータがあった場合はリストにその値をいれていきます
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# 入力とターゲットを入れるための変数を定義します
# また、バランスが取れていないデータを削除していきます
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### 入力の標準化

In [21]:
# sklearnのモジュールを使って前処理をしていきます
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### データのシャッフル

In [22]:
# データをバッチに分けていくことから、何らかの規則性が生まれないようにデータをシャッフルしていきます
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# シャッフルしたデータを変数に入れていきます
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### データセットの分割

In [24]:
# データの数を数えます
samples_count = shuffled_inputs.shape[0]

# 80-10-10に分けることを前提として、訓練用と検証用のデータの数を定義していきます
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# テスト用のデータの数を定義します
test_samples_count = samples_count - train_samples_count - validation_samples_count

# 訓練用データを作成します
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# 検証用データを作成します
validation_inputs = shuffled_inputs[
    train_samples_count : train_samples_count + validation_samples_count
]
validation_targets = shuffled_targets[
    train_samples_count : train_samples_count + validation_samples_count
]

# テスト用データを作成します
test_inputs = shuffled_inputs[train_samples_count + validation_samples_count :]
test_targets = shuffled_targets[train_samples_count + validation_samples_count :]


# 作成したデータの中の１と０の割合を確認します
print(
    np.sum(train_targets),
    train_samples_count,
    np.sum(train_targets) / train_samples_count,
)
print(
    np.sum(validation_targets),
    validation_samples_count,
    np.sum(validation_targets) / validation_samples_count,
)
print(
    np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count
)

1779.0 3579 0.49706621961441744
226.0 447 0.5055928411633109
232.0 448 0.5178571428571429


### データを*.npz形式で保存します

In [25]:
# Ssavezメソッドを使ってデータを保存していきます
np.savez("Audiobooks_data_train", inputs=train_inputs, targets=train_targets)
np.savez(
    "Audiobooks_data_validation", inputs=validation_inputs, targets=validation_targets
)
np.savez("Audiobooks_data_test", inputs=test_inputs, targets=test_targets)

### モデルの作成

In [30]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
        tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
        tf.keras.layers.Dense(output_size, activation="softmax"),
    ]
)

# 損失関数と目的関数を定義します
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# バッチサイズを定義します
batch_size = 100

# エポック数を定義します
max_epochs = 100

# 検証用データを使うことで、過学習を防ぎます
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# モデルをフィットさせます
model.fit(
    train_inputs,
    train_targets,
    batch_size=batch_size,
    epochs=max_epochs,
    callbacks=[early_stopping],
    validation_data=(validation_inputs, validation_targets),
    verbose=2,
)

Epoch 1/100
36/36 - 0s - loss: 0.5434 - accuracy: 0.7214 - val_loss: 0.4919 - val_accuracy: 0.7338 - 311ms/epoch - 9ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4421 - accuracy: 0.7745 - val_loss: 0.4397 - val_accuracy: 0.7718 - 32ms/epoch - 883us/step
Epoch 3/100
36/36 - 0s - loss: 0.4010 - accuracy: 0.7983 - val_loss: 0.4134 - val_accuracy: 0.7562 - 30ms/epoch - 845us/step
Epoch 4/100
36/36 - 0s - loss: 0.3761 - accuracy: 0.8072 - val_loss: 0.3999 - val_accuracy: 0.7785 - 32ms/epoch - 886us/step
Epoch 5/100
36/36 - 0s - loss: 0.3624 - accuracy: 0.8094 - val_loss: 0.3980 - val_accuracy: 0.7785 - 33ms/epoch - 911us/step
Epoch 6/100
36/36 - 0s - loss: 0.3532 - accuracy: 0.8131 - val_loss: 0.3954 - val_accuracy: 0.7875 - 32ms/epoch - 880us/step
Epoch 7/100
36/36 - 0s - loss: 0.3484 - accuracy: 0.8083 - val_loss: 0.3935 - val_accuracy: 0.7830 - 32ms/epoch - 891us/step
Epoch 8/100
36/36 - 0s - loss: 0.3436 - accuracy: 0.8134 - val_loss: 0.3989 - val_accuracy: 0.7696 - 31ms/epoch - 871us/step
E

### モデルのテスト

In [31]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 765us/step - loss: 0.3256 - accuracy: 0.8371


In [32]:
print("\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%".format(test_loss, test_accuracy * 100.0))


Test loss: 0.33. Test accuracy: 83.71%
